### 22232
### Pavan Pyla
### Flan T5

In [1]:
%pip install --upgrade pip -q
%pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet
                                                         ### installing required modules
%pip install \
    transformers==4.27.2 \
    datasets==2.11.0 \
    evaluate==0.4.0 \
    rouge_score==0.1.2 \
    loralib==0.1.1 \
    peft==0.3.0 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 100.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 67.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 76.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.0.2+cu118 requires torch==2.0.1, but you have torch 1.13.1 which is incompatible.
torchtext 0.15.2 requires torch==2.0.1, but you have torch 1.13.1 which is incompatible.
torchtext 0.15.2 requires torchdata==0.6.1, but you have torchdata 0.5.1 whic

In [2]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

In [3]:
huggingface_dataset_name = "databricks/databricks-dolly-15k"

dataset = load_dataset(huggingface_dataset_name)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/databricks___json/databricks--databricks-dolly-15k-7427aa6e57c34282/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
model_name='google/flan-t5-base'                   ## downloadeing the model

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

(…)le/flan-t5-base/resolve/main/config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

(…)base/resolve/main/generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

(…)-base/resolve/main/tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

(…)flan-t5-base/resolve/main/tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

(…)ase/resolve/main/special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [5]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()                         #pprinting the total no of parameters
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(original_model))

trainable model parameters: 247577856
all model parameters: 247577856
percentage of trainable model parameters: 100.00%


In [6]:
from datasets import Dataset
                                                                   # defining a train and test chunk sizes
data = dataset['train'].train_test_split(test_size= 0.4)
data_t = data['test'].train_test_split(test_size= 0.5)
train_data = data['train']
val_data = data_t['test']
test_data = data_t['train']


###  Zero Shot Testing


In [7]:
index = 200

instruction = test_data[index]['instruction']
response = test_data[index]['response']

prompt = f"""
Answer the following question.

{instruction}

Response:                                                           # without trainng , we are trying the zero shot learning
"""

inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"],
        max_new_tokens=200,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN response:\n{response}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Answer the following question.

What did the CFO do after Paypal?

Response:                                                           # without trainng , we are trying the zero shot learning

---------------------------------------------------------------------------------------------------
BASELINE HUMAN response:
Roelof Botha, one of the former CFOs, became partner and Senior Steward of venture capital firm Sequoia Capital. Ken Howery, the other CFO, became a partner at Founders Fund.

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
renamed adobe acquisition


<a name='2'></a>
## 2 - Perform Full Fine-Tuning

In [8]:
def tokenize_function(example):
    start_prompt = 'Respond to the following Instructions.\n\n'
    end_prompt = '\n\nResponse: '
    prompt = [start_prompt + instruction + end_prompt for instruction in example["instruction"]]
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example["response"], padding="max_length", truncation=True, return_tensors="pt").input_ids

    return example
                                       # tokenizing the data in format (\s + instruction +\s) for every instruction

tokenized_datasets_train = train_data.map(tokenize_function, batched=True)
tokenized_datasets_train = tokenized_datasets_train.remove_columns([ 'instruction', 'context', 'response', 'category',])

tokenized_datasets_val = val_data.map(tokenize_function, batched=True)
tokenized_datasets_val = tokenized_datasets_val.remove_columns([ 'instruction', 'context', 'response', 'category',])

tokenized_datasets_test = test_data.map(tokenize_function, batched=True)
tokenized_datasets_test = tokenized_datasets_test.remove_columns([ 'instruction', 'context', 'response', 'category',])

Map:   0%|          | 0/9006 [00:00<?, ? examples/s]

Map:   0%|          | 0/3003 [00:00<?, ? examples/s]

Map:   0%|          | 0/3002 [00:00<?, ? examples/s]

In [9]:
tokenized_datasets_train = tokenized_datasets_train.filter(lambda example, index: index % 100 == 0, with_indices=True)
tokenized_datasets_val = tokenized_datasets_val.filter(lambda example, index: index % 100 == 0, with_indices=True)                      #dividing the tokenized data into in train and test
tokenized_datasets_test = tokenized_datasets_test.filter(lambda example, index: index % 100 == 0, with_indices=True)

Filter:   0%|          | 0/9006 [00:00<?, ? examples/s]

Filter:   0%|          | 0/3003 [00:00<?, ? examples/s]

Filter:   0%|          | 0/3002 [00:00<?, ? examples/s]

In [10]:
print(f"Shapes of the datasets:")
print(f"Training: {tokenized_datasets_train.shape}")
print(f"Validation: {tokenized_datasets_val.shape}")
print(f"Test: {tokenized_datasets_test.shape}")                    # checking their shapes

print(tokenized_datasets_train)
print(tokenized_datasets_val)
print(tokenized_datasets_test)

Shapes of the datasets:
Training: (91, 2)
Validation: (31, 2)
Test: (31, 2)
Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 91
})
Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 31
})
Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 31
})


In [11]:
output_dir = f'./instruction-response-training-{str(int(time.time()))}'

training_args = TrainingArguments(
    output_dir= output_dir,
    overwrite_output_dir= True,
    learning_rate= 1e-5,
    num_train_epochs= 1,
    weight_decay= 0.01,
    logging_steps= 1,
    max_steps= 1,
    # save_total_limit= 1,
    save_strategy = "steps",
    # load_best_model_at_end= False,
    evaluation_strategy="steps",
)

trainer = Trainer(
    model= original_model,
    args= training_args,
    train_dataset= tokenized_datasets_train,
    eval_dataset= tokenized_datasets_val,
)

In [12]:
trainer.train()                  ## training the model

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss
1,42.000000,36.991936


TrainOutput(global_step=1, training_loss=42.0, metrics={'train_runtime': 9.0262, 'train_samples_per_second': 0.886, 'train_steps_per_second': 0.111, 'total_flos': 5478058819584.0, 'train_loss': 42.0, 'epoch': 0.08})

In [13]:
trainer.save_model('ins_model')   ## saving the model

In [14]:
instruct_model = AutoModelForSeq2SeqLM.from_pretrained("/content/ins_model", torch_dtype=torch.bfloat16)

In [15]:
original_model = original_model.to('cuda')
instruct_model = instruct_model.to('cuda')

In [16]:
index = 200
instruction = test_data[index]['instruction']
human_baseline_response = test_data[index]['response']

prompt = f"""
Answer the following question.

{instruction}

Response:
"""

input_ids = tokenizer(prompt, return_tensors="pt").input_ids
input_ids = input_ids.to('cuda')

original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

print(dash_line)
print(f'BASELINE HUMAN RESPONSE:\n{human_baseline_response}')
print(dash_line)
print(f'ORIGINAL MODEL:\n{original_model_text_output}')
print(dash_line)
print(f'INSTRUCT MODEL:\n{instruct_model_text_output}')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN RESPONSE:
Roelof Botha, one of the former CFOs, became partner and Senior Steward of venture capital firm Sequoia Capital. Ken Howery, the other CFO, became a partner at Founders Fund.
---------------------------------------------------------------------------------------------------
ORIGINAL MODEL:
restructured
---------------------------------------------------------------------------------------------------
INSTRUCT MODEL:
restructured


In [17]:
rouge = evaluate.load('rouge')

In [18]:
instructions = test_data[0:10]['instruction']
human_baseline_responses = test_data[0:10]['response']

original_model_responses = []
instruct_model_responses = []

for _, instruction in enumerate(instructions):
    prompt = f"""
                Answer the following question.

                {instruction}

                Response: """
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    input_ids = input_ids.to('cuda')

    original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)
    original_model_responses.append(original_model_text_output)

    instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)
    instruct_model_responses.append(instruct_model_text_output)

zipped_responses = list(zip(human_baseline_responses, original_model_responses, instruct_model_responses))

df = pd.DataFrame(zipped_responses, columns = ['human_baseline_responses', 'original_model_responses', 'instruct_model_responses'])
df

,human_baseline_responses,original_model_responses,instruct_model_responses
0,If you watched any of the Women's March Madnes...,sarah harris,sarah harris
1,Portsmouth is a coastal town in the south of E...,pier,pier
2,"Visual Basic (VB), originally called Visual Ba...",programming language,programming language
3,C. Shannon Bacon (born 1971/1972) is an Americ...,a sailor,a sailor
4,"Green Turtle is alive, Hovasaurus is extinct.",tiger,tiger
5,"Scarlett Johansson was born November 22, 1984.",19 September 1977,19 September 1977
6,"The saying is ""The early bird gets the worm""",swans,swans
7,Notable California regions that produce pinot ...,California,California
8,Apache Spark is a multi-language engine for ex...,software,software
9,"Judaism, Christianity and Islam are the three ...",judaism and christianity,judaism and christianity


Evaluate the models computing ROUGE metrics. Notice the improvement in the results!

In [19]:
original_model_results = rouge.compute(
    predictions=original_model_responses,
    references=human_baseline_responses[0:len(original_model_responses)],
    use_aggregator=True,
    use_stemmer=True,
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_responses,
    references=human_baseline_responses[0:len(instruct_model_responses)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)

ORIGINAL MODEL:
{'rouge1': 0.05404067197170646, 'rouge2': 0.0015384615384615385, 'rougeL': 0.03727674624226347, 'rougeLsum': 0.03865605658709107}
INSTRUCT MODEL:
{'rouge1': 0.05404067197170646, 'rouge2': 0.0015384615384615385, 'rougeL': 0.03727674624226347, 'rougeLsum': 0.03865605658709107}


The results show substantial improvement in all ROUGE metrics:

In [20]:
print("Absolute percentage improvement of INSTRUCT MODEL over ORIGINAL MODEL")

improvement = (np.array(list(instruct_model_results.values())) - np.array(list(original_model_results.values())))
for key, value in zip(instruct_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

Absolute percentage improvement of INSTRUCT MODEL over ORIGINAL MODEL
rouge1: 0.00%
rouge2: 0.00%
rougeL: 0.00%
rougeLsum: 0.00%


<a name='3'></a>
## 3 - Perform Parameter Efficient Fine-Tuning (PEFT)


<a name='3.1'></a>
### 3.1 - Setup the PEFT/LoRA model for Fine-Tuning

You need to set up the PEFT/LoRA model for fine-tuning with a new layer/parameter adapter. Using PEFT/LoRA, you are freezing the underlying LLM and only training the adapter. Have a look at the LoRA configuration below. Note the rank (`r`) hyper-parameter, which defines the rank/dimension of the adapter to be trained.

In [21]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=32, # Rank
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM # FLAN-T5
)

Add LoRA adapter layers/parameters to the original LLM to be trained.

In [22]:
peft_model = get_peft_model(original_model,
                            lora_config)
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 3538944
all model parameters: 251116800
percentage of trainable model parameters: 1.41%


<a name='3.2'></a>
### 3.2 - Train PEFT Adapter

Define training arguments and create `Trainer` instance.

In [23]:
torch.cuda.empty_cache()

In [24]:
output_dir = f'./peft-instruction-response-training-{str(int(time.time()))}'

peft_training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    learning_rate=1e-3, # Higher learning rate than full fine-tuning.
    num_train_epochs=5,
    logging_steps=5,
    max_steps=1,
    save_strategy = "no",
    load_best_model_at_end= False,
)

peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_datasets_train,
)

Now everything is ready to train the PEFT adapter and save the model.

In [25]:
peft_trainer.train()

peft_model_path="./peft-instruction-response-checkpoint-local"

peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


('./peft-instruction-response-checkpoint-local/tokenizer_config.json',
 './peft-instruction-response-checkpoint-local/special_tokens_map.json',
 './peft-instruction-response-checkpoint-local/tokenizer.json')

That training was performed on a subset of data. To load a fully trained PEFT model, read a checkpoint of a PEFT model from S3.

Check that the size of this model is much less than the original LLM:

Prepare this model by adding an adapter to the original FLAN-T5 model. You are setting `is_trainable=False` because the plan is only to perform inference with this PEFT model. If you were preparing the model for further training, you would set `is_trainable=True`.

In [26]:
torch.cuda.empty_cache()

In [27]:
from peft import PeftModel, PeftConfig

peft_model_base = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base", torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

peft_model = PeftModel.from_pretrained(peft_model_base,
                                       '/content/peft-instruction-response-checkpoint-local',
                                       torch_dtype=torch.bfloat16,
                                       is_trainable=False)

In [28]:
peft_model = peft_model.to('cuda')

The number of trainable parameters will be `0` due to `is_trainable=False` setting:

In [29]:
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 0
all model parameters: 251116800
percentage of trainable model parameters: 0.00%


<a name='3.3'></a>
### 3.3 - Evaluate the Model Qualitatively (Human Evaluation)

Make inferences for the same example as in sections [1.3](#1.3) and [2.3](#2.3), with the original model, fully fine-tuned and PEFT model.

In [30]:
index = 200
instruction = test_data[index]['instruction']
baseline_human_response = test_data[index]['response']

prompt = f"""
Answer the following question.

{instruction}

response: """

input_ids = tokenizer(prompt, return_tensors="pt").input_ids
input_ids = input_ids.to('cuda')

original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

print(dash_line)
print(f'BASELINE HUMAN response:\n{human_baseline_response}')
print(dash_line)
print(f'ORIGINAL MODEL:\n{original_model_text_output}')
print(dash_line)
print(f'INSTRUCT MODEL:\n{instruct_model_text_output}')
print(dash_line)
print(f'PEFT MODEL:\n{peft_model_text_output}')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN response:
Roelof Botha, one of the former CFOs, became partner and Senior Steward of venture capital firm Sequoia Capital. Ken Howery, the other CFO, became a partner at Founders Fund.
---------------------------------------------------------------------------------------------------
ORIGINAL MODEL:
merged with a new company
---------------------------------------------------------------------------------------------------
INSTRUCT MODEL:
ad hoc
---------------------------------------------------------------------------------------------------
PEFT MODEL:
ad hoc



### 3.4 - Evaluate the Model with ROUGE Metric


In [31]:
instructions = test_data[0:10]['instruction']
human_baseline_responses = test_data[0:10]['response']

original_model_responses = []
instruct_model_responses = []
peft_model_responses = []

for idx, instruction in enumerate(instructions):
    prompt = f"""
Answer the following question.

{instruction}

response: """

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    input_ids = input_ids.to('cuda')

    human_baseline_text_output = human_baseline_responses[idx]

    original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

    instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

    peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

    original_model_responses.append(original_model_text_output)
    instruct_model_responses.append(instruct_model_text_output)
    peft_model_responses.append(peft_model_text_output)

zipped_responses = list(zip(human_baseline_responses, original_model_responses, instruct_model_responses, peft_model_responses))

df = pd.DataFrame(zipped_responses, columns = ['human_baseline_responses', 'original_model_responses', 'instruct_model_responses', 'peft_model_responses'])
df

,human_baseline_responses,original_model_responses,instruct_model_responses,peft_model_responses
0,If you watched any of the Women's March Madnes...,jim wilson,sarah harris,sarah harris
1,Portsmouth is a coastal town in the south of E...,Great harbour,pier,pier
2,"Visual Basic (VB), originally called Visual Ba...",Visual Basic,programming language,programming language
3,C. Shannon Bacon (born 1971/1972) is an Americ...,a spokesman for the swiss federation,a sailor,a sailor
4,"Green Turtle is alive, Hovasaurus is extinct.",tiger,tiger,tiger
5,"Scarlett Johansson was born November 22, 1984.",january 18,1939,1939
6,"The saying is ""The early bird gets the worm""",twiggy,swans,swans
7,Notable California regions that produce pinot ...,"san antonio, san antonio, san antonio, san ant...",California,California
8,Apache Spark is a multi-language engine for ex...,a software application,software,software
9,"Judaism, Christianity and Islam are the three ...",islam,judaism and christianity,judaism and christianity


In [32]:
rouge = evaluate.load('rouge')

original_model_results = rouge.compute(
    predictions=original_model_responses,
    references=human_baseline_responses[0:len(original_model_responses)],
    use_aggregator=True,
    use_stemmer=True,
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_responses,
    references=human_baseline_responses[0:len(instruct_model_responses)],
    use_aggregator=True,
    use_stemmer=True,
)

peft_model_results = rouge.compute(
    predictions=peft_model_responses,
    references=human_baseline_responses[0:len(peft_model_responses)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)
print('PEFT MODEL:')
print(peft_model_results)

ORIGINAL MODEL:
{'rouge1': 0.031550945698596705, 'rouge2': 0.0015384615384615385, 'rougeL': 0.02921212121212121, 'rougeLsum': 0.031550945698596705}
INSTRUCT MODEL:
{'rouge1': 0.05404067197170646, 'rouge2': 0.0015384615384615385, 'rougeL': 0.03727674624226347, 'rougeLsum': 0.03865605658709107}
PEFT MODEL:
{'rouge1': 0.05404067197170646, 'rouge2': 0.0015384615384615385, 'rougeL': 0.03727674624226347, 'rougeLsum': 0.03865605658709107}


Notice, that PEFT model results are not too bad, while the training process was much easier!

In [33]:
results = df.copy()
human_baseline_responses = results['human_baseline_responses'].values
original_model_responses = results['original_model_responses'].values
instruct_model_responses = results['instruct_model_responses'].values
peft_model_responses     = results['peft_model_responses'].values

original_model_results = rouge.compute(
    predictions=original_model_responses,
    references=human_baseline_responses[0:len(original_model_responses)],
    use_aggregator=True,
    use_stemmer=True,
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_responses,
    references=human_baseline_responses[0:len(instruct_model_responses)],
    use_aggregator=True,
    use_stemmer=True,
)

peft_model_results = rouge.compute(
    predictions=peft_model_responses,
    references=human_baseline_responses[0:len(peft_model_responses)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)
print('PEFT MODEL:')
print(peft_model_results)

ORIGINAL MODEL:
{'rouge1': 0.031550945698596705, 'rouge2': 0.0015384615384615385, 'rougeL': 0.02921212121212121, 'rougeLsum': 0.031550945698596705}
INSTRUCT MODEL:
{'rouge1': 0.05404067197170646, 'rouge2': 0.0015384615384615385, 'rougeL': 0.03727674624226347, 'rougeLsum': 0.03865605658709107}
PEFT MODEL:
{'rouge1': 0.05404067197170646, 'rouge2': 0.0015384615384615385, 'rougeL': 0.03727674624226347, 'rougeLsum': 0.03865605658709107}


In [34]:
print("Absolute percentage improvement of PEFT MODEL over ORIGINAL MODEL")

improvement = (np.array(list(peft_model_results.values())) - np.array(list(original_model_results.values())))
for key, value in zip(peft_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

Absolute percentage improvement of PEFT MODEL over ORIGINAL MODEL
rouge1: 2.25%
rouge2: 0.00%
rougeL: 0.81%
rougeLsum: 0.71%


In [35]:
print("Absolute percentage improvement of PEFT MODEL over INSTRUCT MODEL")

improvement = (np.array(list(peft_model_results.values())) - np.array(list(instruct_model_results.values())))
for key, value in zip(peft_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

Absolute percentage improvement of PEFT MODEL over INSTRUCT MODEL
rouge1: 0.00%
rouge2: 0.00%
rougeL: 0.00%
rougeLsum: 0.00%
